[//]: # (Image References)

[image1]:learning_curves.png "Learning Curves for Proximal Policy Optimization with Generalized Advantage Estimation"

# Report: Collaboration and Competition Project

This work trains a pair of agents to play a simulated game of tennis (or something similar to tennis) with the objective of collaboratively keeping the ball in the air for as long as possible. The environment has a continuous state space, representing positions and velocities of the rackets and ball, as well as a continuous action space, representing the acceleration of the rackets. Each agent is controlled separately and only has access to it's own "local" observation of the environment; i.e. each agent can observe it's own position and velocity as well as the ball's position and velocity, but not the position and velocity of the other agent. To accelerate training in this collaborative environment, a single policy is trained but used by both agents.

The reinforcement learning algorithm used in this work is [proximal policy optimization (PPO)](https://arxiv.org/pdf/1707.06347.pdf) in an actor-critic form and a [generalized advantage estimate (GAE)](https://arxiv.org/pdf/1506.02438.pdf). Two deep neural networks are trained: one for the stochastic policy that maps the robots state to actions, and one for the value function that estimates the expected returns from a given state of the robot. 


## Learning Algorithm

As previously mentioned, this work uses PPO for policy training and GAE for advantage estimation. This is an actor-critic method with two neural networks: one for the policy and one for the value function.

The algorithm works by rolling out a fixed policy for a fixed number of timesteps to generate a batch of training data. Each batch is composed of $NT$ timesteps of data from the $N=2$ collaborative tennis agents being executed for $T$ timesteps. After a rollout, the empirical returns, value estimates, and advantage estimates are calculated for every timestep of the training batch. The advantage estimates are used to compute the policy loss and update the policy parameters $\theta$. The returns and value estimates are then used to compute the value loss and update the value function parameters $w$.

#### Policy Loss Function
Proximal policy optimization learns by minimization a surrogate loss function, $L^{\text{CLIP}+\text{S}}$, over the policy parameters $\theta$ based on a batch of training data. The loss function is of the form

\begin{equation}
L \left( \theta \right) = \mathbb{E} \left[ \frac{\pi(a_t \mid s_t, \theta) }{\pi_{\text{old}}(a_t \mid s_t, \theta)} A^{\text{GAE}}(s_t,a_t)\right] = \mathbb{E} \left[ \rho_t (\theta) A_t\right]
\end{equation}

\begin{equation}
L^{\text{CLIP}+\text{S}} \left( \theta \right) = - \mathbb{E} \left[ \min{\left( \rho_t (\theta) A_t, \text{clip}\left(\rho_t(\theta), 1-\epsilon, 1+\epsilon\right) \right)} + \beta S \right] 
\end{equation}

where $s$ is the state of the system, $a$ is the action the agent takes, $S$ is a entropy bonus to encourage exploration weighted by the hyperparameter $\beta$.

#### Value Loss Function
The advantage estimate $A_t$ is computed as 

\begin{align}
A_t &= \sum_{l=0}^{\infty} \left( \gamma \lambda \right)^l \delta_{t+l}^{V} \\
\delta_{t}^{V} &= r_t + \gamma V_w (s_{t+1}) - V_w (s_t)
\end{align}

Where the value estimates $V_w (s_t)$ are generated by the value network which is trained by minimizing over value parameters $w$ the squared error with respect the empirical returns from a training batching; i.e. minimizing

\begin{equation}
L^V (w) = \left(V_w (s_t) - G_t \right)^{2}
\end{equation}

### Execution Script: `training_script.py`

The python script `training_script.py` is effectively the "main" function for the code. An example call of the script is given as:
```bash
python training_script.py --exp-name my_experiment 
```
`training_script.py` is responsible for establishing the environment (i.e. `unityagents.UnityEnvironment`), creating the agents that interact with the environment (i.e. `actor_critic_mind.ActorCriticMind`), and executing the training for various agents (i.e. `trainer.py`).

Alternatively, one can work through the instructions in `Tennis.ipynb`


### Source Code and Model Architecture 

The python module `continuous_control_brain_agent.py` contains most of the source code run by `Continuous_Control.ipynb`. The `continuous_control_brain_agent` module provides a class for defining trainable agents (`ActorCriticMind`) and a functions for stepping through the training processes (`train_actor_critic_mind` and `collect_trajectories`) as well as functions for computing the policy loss (`clipped_surrogate_objective`). 

The `ActorCriticMind` class defines objects for storing and training the agents that interact with the environment. The policy for selecting actions from a continuous action space given an observation of the environment is stochastic. The output layer of the policy generates outputs parameters for a normal distribution for each dimension of the action space. These distributions are then randomly sampled in order to selecting the action that the agent applies to the environment. The policy function is stored in `ActorCriticMind.policy_network` and is a object of type `network_models.GaussianActorNetwork` defined in the `network_models.py` module. The `ActorCriticMind.policy_network` action value function is a deep neural network defined using PyTorch.

Similarly the `ActorCriticMind.value_network` object of type `network_models.DeepNetwork` stores a deep neural network for computing an estimated expected return given an environment state. The 

The policy network takes input vectors of size 20x33 (i.e. the observation space size for the environment by the number of agents) and outputs vectors of size 4 (i.e. the action space size for the environment). The value network takes in the same input vectors and outputs a single floating point value. For both networks a deep neural net is used with fully-connected hidden layers of size 256, 64, and 32, respectively. Each layer uses a ReLU activation function.

As the `train_actor_critic_mind` calls the `collect_trajectories` function that rolls out the policy in the environment for a predetermined number of steps or until the end of an episode. The `train_actor_critic_mind` function than uses the batch of data to train the agent's policy and value networks by breaking the batch into randomized minibatches, computing policy and value loss for each minibatches; repeating the process over multiple epochs per batch. 

### Data Files

Results from training are stored in `.pth` and `.pkl` files. The `.pth` are based on PyTorch's [`save`](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save) function and stored the model parameters for a trained neural network. The `.pkl` files are [pickle files](https://docs.python.org/3/library/pickle.html) that store the scores of different learning algorithms throughout their training process.

The `utils/training_analysis.py` module defines functions for plotting and visualizing the training results. This module is designed so that it can be used in the `Navigation.ipynb` notebook as well as from command line. For example:

```bash
python training_analysis.py my_experiment_scores.pkl
```

will render a plot of the learning curves and loss curves based on training data stored in the respective `.pkl` pickle files. The ability to render plots from command line using saved pickle files makes it easier and faster to iterate on the post-processing scripts without re-running training.

### Hyperparameters

The following hyperparameters were used during training:

+ n_episodes = 200        # number of episodes
+ batch_size = 64         # batch size
+ minibatch_size = 16     # minibatch size
+ n_epochs = 4            # number training epochs per batch
+ gamma = 0.9             # discount factor
+ lambda = 0.5            # GAE parameter
+ policy_lr = 2e-4        # policy learning rate
+ value_lr = 1e-3         # value learning rate
+ beta_start = 0.1        # initial entropy exploration factor
+ beta_end=0.0005         # final entropy exploration factor
+ beta_decay=0.99         # decay entropy exploration factor
+ epsilon_start = 0.3     # initial policy clipping factor
+ epsilon_end=0.1         # final policy clipping factor
+ epsilon_decay=0.99      # decay rate of policy clipping factor
+ v_epsilon_start = 0.3   # initial value clipping factor
+ v_epsilon_end=0.1       # final value clipping factor
+ v_epsilon_decay=0.99    # decay rate of value clipping factor

## Plot of Rewards

<img src="learning_curves.png">

On the above plot we see the learning curves for 3 different $\lambda$ values: 0.1, 0.5, and 0.9. The lines show the moving average over the past 10 episodes and the shade regions show the standard deviation. For the best training performance, the learning curve asymptotes within 25 episodes. This results in the environment being considered "solved" in a matter of 3-4 episodes since the environment solution is measured as an average over 100 episodes.

The $\lambda$ value acts as a blending factor between temporal difference (TD) and Monte Carlo methods. At $\lambda=1$, GAE is equivalent to Monte Carlo, which is an unbiased estimate of the true returns but suffers from high variance. At $\lambda = 0$, GAE is equivalent to TD which reduces variance by introduces bias. 

The plot shows that all lambda values are able to learn effectively in this environment with lower lambda values learning more quickly. $\lambda=0.1$ shows the best performance with $\lambda=0.5$ learning slightly slower. $\lambda=0.9$ takes roughly twice as many episodes to reach final performance.

## Ideas for Future Work

This approach has shown very rapid training for this environment; therefore there is relatively little room for improvement. Future work could entail comparison between other algorithms such as DDPG, A2C, or others.